In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from tqdm import tqdm
import numpy as np

In [23]:
device = torch.device("mps")

In [41]:
#encode the alphabet into one-hot tensors
def encode_letter(letter):
    vector = torch.zeros(1, 26)
    vector[0][ord(letter) - 97] = 1
    return vector

def decode_letter(vector):
    return chr(torch.argmax(vector).item() + 97)

data = []
labels = []
for i in range(26):
    data.append(encode_letter(chr(i + 97)))
    labels.append(encode_letter(chr(i + 97)))

In [208]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        layers = [26, 150, 64, 3, 64, 150, 26]
        # Define the layers
        self.fc1 = nn.Linear(layers[0], layers[1])
        self.fc2 = nn.Linear(layers[1], layers[2])
        self.fc3 = nn.Linear(layers[2], layers[3])
        self.fc4 = nn.Linear(layers[3], layers[4])
        self.fc5 = nn.Linear(layers[4], layers[5])
        self.fc6 = nn.Linear(layers[5], layers[6])

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
    def encode(self, x):
        x_chars = []
        for i in range(len(x)):
            x_chars.append(encode_letter(x[i]))
        out = []
        for i in range(len(x_chars)):
            x = x_chars[i]
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            out.append(x)
        return torch.stack(out)
    def decode(self, x):
        out = []
        for ten in x:
            ten = F.relu(self.fc4(ten))
            ten = F.relu(self.fc5(ten))
            ten = self.fc6(ten)
            out.append(decode_letter(ten))
        return ''.join(out)
loss_function = nn.MSELoss()
model = NeuralNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True, min_lr=1e-6)


In [209]:
print("Total number of parameters: ", sum(p.numel() for p in model.parameters()))

Total number of parameters:  27841


In [193]:
def efficiency_test():
    num_wrong = 0
    for i in range(int(26)):
        let = chr(i + 97)
        letter = encode_letter(let)
        output = model(letter)
        output = output.detach().numpy()
        output = np.argmax(output)
        if output != ord(let) - 97:
            num_wrong += 1
    return num_wrong

print(efficiency_test())

25


In [ ]:
max_epochs = 2000
epoch = 0
while efficiency_test() != 0 and epoch < max_epochs:
    epoch += 1
    epoch_loss = 0
    for i in range(len(data)):
        model.zero_grad()
        output = model(data[i])
        loss = loss_function(output, labels[i])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    scheduler.step(epoch_loss/len(data))
    if epoch % 100 == 0:
        print('Epoch: ', epoch, 'Loss: ', loss.item(), 'Letters Wrong:', efficiency_test())

In [ ]:
model.decode(model.encode("abcdefghijklmnopqrstuvwxyz"))

'abcdefghijklmnopqrstuvwxyz'